In [313]:
#pip install pandas openpyxl 

In [314]:
import pandas as pd
import numpy as np
import re
import unicodedata

**Import datasets**

In [315]:
# Woman
df1 = pd.read_csv('/Users/roraimachavez/Downloads/7.IRONHACK/Projects/final_project/data/woman/woman.csv')
df2 = pd.read_csv('/Users/roraimachavez/Downloads/7.IRONHACK/Projects/final_project/data/woman/woman1.csv')
df3 = pd.read_csv('/Users/roraimachavez/Downloads/7.IRONHACK/Projects/final_project/data/woman/woman2.csv')
df4 = pd.read_csv('/Users/roraimachavez/Downloads/7.IRONHACK/Projects/final_project/data/woman/woman3.csv')
df5 = pd.read_csv('/Users/roraimachavez/Downloads/7.IRONHACK/Projects/final_project/data/woman/woman4.csv')

In [316]:
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

Add “profile” column to indicate whether the profile is female or male.

In [317]:
df['profile'] = 'woman'

In [318]:
# Man
#df7 = pd.read_csv('/Users/roraimachavez/Downloads/7.IRONHACK/Projects/final_project/data/woman/woman.csv')

In [319]:
#df_man = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

Add “profile” column to indicate whether the profile is female or male.

In [320]:
#df_man['profile'] = 'man'

In [321]:
#df = pd.concat([df_woman, df_man], ignore_index=True)

In [322]:
df.head()

,name,age,location,story_about,job,education,heightv2,intentionsv2,exercisev2,educationv2,drinkingv2,smokingv2,familyPlansv2,religionv2,Politicsv2,genderv2,starSignv2,profile
0,Tarek,26,"Sant Cugat del Vallès, Barcelona",NaN,NaN,NaN,175 cm,Something casual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,woman
1,Guillem,41,Barcelona,Philosophy and film studies.\n\nFrom electroni...,Film Editor And Teacher,UAB-UB-EDC,184 cm,Something casual,Sometimes,Graduate degree,Socially,Socially,Not sure yet,Atheist,Left,NaN,NaN,woman
2,Bernardo,31,Barcelona,NaN,NaN,NaN,180 cm,Something casual,Sometimes,Graduate degree,Socially,Socially,Not sure yet,Atheist,Left,Man,NaN,woman
3,Leonardo,28,Barcelona,Looking for company to explore the World,Electrical Engineer,UFSC,174 cm,NaN,Sometimes,Undergraduate degree,Socially,Never,Not sure yet,NaN,Apolitical,Man,Aquarius,woman
4,Ahmed,31,Barcelona,NaN,NaN,NaN,186 cm,Relationship,Sometimes,Graduate degree,Socially,Socially,Not sure yet,NaN,Apolitical,Man,Scorpio,woman


In [323]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           1020 non-null   object
 1   age            1020 non-null   int64 
 2   location       1006 non-null   object
 3   story_about    792 non-null    object
 4   job            547 non-null    object
 5   education      344 non-null    object
 6   heightv2       1005 non-null   object
 7   intentionsv2   967 non-null    object
 8   exercisev2     1000 non-null   object
 9   educationv2    934 non-null    object
 10  drinkingv2     1003 non-null   object
 11  smokingv2      944 non-null    object
 12  familyPlansv2  836 non-null    object
 13  religionv2     670 non-null    object
 14  Politicsv2     540 non-null    object
 15  genderv2       984 non-null    object
 16  starSignv2     913 non-null    object
 17  profile        1020 non-null   object
dtypes: int64(1), object(17)
memo

Change columns names

In [324]:
new_column_names = {
    'story_about' : 'about_me',
    'heightv2': 'height',
    'intentionsv2': 'intentions',
    'exercisev2': 'exercise',
    'educationv2': 'educationTag',
    'drinkingv2': 'drinking',
    'smokingv2': 'smoking',
    'familyPlansv2': 'childrens',
    'religionv2': 'religion',
    'Politicsv2': 'politics',
    'genderv2': 'gender',
    'starSignv2': 'zodiacSign',
}

df.rename(columns=new_column_names, inplace=True)

Remove "cm" in column height so is a int number

In [325]:
df['height'] = df['height'].str.replace(' cm', '')

Remove "Barcelona" in column location so I only take the town

In [326]:
df['location'] = df['location'].str.split(',').str[0]

Looking for duplicate column

In [327]:
duplicated_rows = df[df.duplicated()]
len(duplicated_rows) 

0

Use regex to make the job column more available

In [328]:
# Apply unidecode to each DataFrame element to remove accents.
df = df.applymap(lambda x: ''.join([c for c in unicodedata.normalize('NFKD', x) if not unicodedata.combining(c)]) if isinstance(x, str) else x)

/var/folders/__/k53x5x752318rh35q54k32m80000gn/T/ipykernel_38061/3211241325.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join([c for c in unicodedata.normalize('NFKD', x) if not unicodedata.combining(c)]) if isinstance(x, str) else x)


In [329]:
df["job"].count()

547

In [330]:
#General view of the activities
unicos = df["job"].unique()
for i in unicos:
    print(i)
len(unicos)

nan
Film Editor And Teacher
Electrical Engineer
model
Enginyer informatic
Consultant at Recruitment
Program Manager in IT & Podcaster
Ceo
Account manager
Startup Founder
Padagoge fur Kinder at Kindergarten
Empresario
Comunicacio
Higienista Bucodental
Growth Director
Compositor musical at free worker
Pizzaiolo at Alma Mia
La Vida
Medico
Business Manager at Tecnologia
Ingeniero de Software
Professor
Software Engineer at Moving Picture Company
Infectious Diseases Medical Doctor
General partner at Encomenda Venture Capital
Sales team leader Catalunya at TheFork
Software Engineer
Project Manager at IT
Realtor at Berkshire Hathaway Home Services Golden Properties
Tecnic Electric
Sales at Finland
High school teacher
Chef at Aquamarina Hotel
Country Manager at Tech SaaS
Tecnico en Anatomia Patologica
escritor at zaino
Beca en investigacion at Instituto de Investigacion en IA
Global Marketing Director
Marketing at Aura Empresas
Director at Inteligencia Artificial
buisness-owner
Engineer
Associa

488

In [331]:
dict_job = {
    r"[Pp]iloto|[Pp]iloto": "pilot",
    r"[Ff]arma|[Pp]harmaci": "pharmacisty",
    r"[Aa]ctor|[Mm]odel": "actor",
    r"[Cc]reative|[Cc]reativo|[Aa]rte": "creative director",
    r"[Dd]esign|[Dd]iseñad|[Dd]ecorateur": "design",
    r"[Cc]eo": "ceo",
    r"[Aa]rchitect|[Aa]rquitec": "architect",
    r"[Aa]dministr": "administrator",
    r"[Mm]echanical|[Mm]ecanica": "mechanical engineer",
    r"[Dd]entist|[Oo]dontol|[Bb]ucal|[Dd]ental": "dentist",
    r"[Rr]eal state|[Ii]nmobi": "real state",
    r"[Ff]isioterapeuta|[Pp]hysiotherapist": "physiotherapist",
    r"[Cc]onsult": "consultant",
    r"[Ll]ogistic": "Logistics",
    r"[Tt]elecommunications|[Tt]elecomuni": "telecommunications",
    r"[Ee]dition|3[Dd],[Mm]otion,[Cc]olorist": "editor",
    r"[Ww]eb|[Dd]evelo": "web developer",
    r"[Cc]ontable|[Bb]ank": "accountant",
    r"[Ff]ilm|[Aa]udiovisual|[Aa]v": "film industry",
    r"[Ss]oftware|[Pp]rogramador": "software engineer",
    r"[Aa]cco": "account manager",
    r"[Mm]usic|[Cc]omposit|[Dd]j|[Ss]inger": "music industry",
    r"[Rr]ecruitment|[Hh]R": "recruitment or consultant",
    r"[Ss]ales|[Ss]eller": "sales",
    r"[Pp]roject": "project manager",
    r"[Tt]eacher|[Pp]rofessor|[Pp]edagogo|[Pp]rofeso": "teacher",
    r"[Ww]riter|[Ee]scritor": "writer",
    r"[Ff]ounder|[Ff]undador|[Oo]wner|[Pp]ropietario": "founder/owner",
    r"[Dd]octor|[Mm]edico|[Ss]anidad|[m]etge|[Ii]nfermer|[Ee]nfermero|[Nn]urse": "doctor or nurse",
    r"[Cc]omunic|[Cc]ommuni|[Ll]ocutor": "communication",
    r"[Cc]hef|[Cc]ocinero|[Cc]uiner": "chef",
    r"[Ee]ntrepreneur|[Ee]mpresario|[Bb]uisness|[Bb]usiness": "entrepreneur",
    r"[Mm]arketing|[Cc]ontent|[Cc]opywriter": "marketing",
    r"[Qq]imic|[Qq]uimi|[Cc]hemical": "chemical",
    r"[Ff]inanc": "financial",
    r"[Aa]bogado|[Ll]awyer": "lawyer",
    r"[Aa]arqueologo|[Aa]rchaeologist": "archaeologist",
    r"[Tt]ech": "tech",
    r"[Aa]nalyst|[Aa]nalista|[Dd]ata": "data analyst/scientist",
    r"[Ss]tudent|[Es]tudiante": "student",
    r"[Cc]onstruction|[Cc]onstruc": "construction",
    r"[Ff]otogra|[Pp]hotogra": "photographer",
    r"[Pp]sicologo|[Pp]hotogra|[Tt]herapist": "psychologist",
    r"[Aa]artifi": "AI",
    r"[Rr]esearch|[Rr]esearcher" : "research"}


In [332]:
# Function to categorize species
def categorize_activities(text, dict_job):
    if isinstance(text, str):  # Check if the value is a text string
        for pattern, category in dict_job.items():
            if re.search(pattern, text):
                return category
    return "Other"  # If it does not match any category, "Other" is returned.

In [333]:
# Clear invalid values in the "job" column
df["job"] = df["job"].fillna("empty")  # Fill NaN values with an empty string
df["job"] = df["job"].astype(str)  # Convert all values to text strings

In [334]:
df["job"].value_counts()

job
empty                                     473
Software Engineer                           6
Software Developer                          6
Empresario                                  5
Chef                                        5
                                         ... 
Cozinheira/empresaria                       1
Colorista                                   1
AV at Eventos                               1
Director Sostenibilidad                     1
Creative Designer at Digital Marketing      1
Name: count, Length: 488, dtype: int64

In [335]:
# Apply the function to categorize activities
df["job"] = df["job"].apply(lambda x: categorize_activities(x, dict_job))

In [336]:
count_activity = df["job"].value_counts()
print(count_activity)

job
Other                        689
entrepreneur                  25
tech                          24
software engineer             22
consultant                    18
web developer                 17
music industry                16
founder/owner                 14
design                        14
sales                         13
film industry                 11
marketing                     11
chef                          11
project manager               11
lawyer                        10
data analyst/scientist        10
teacher                        8
creative director              8
doctor or nurse                8
account manager                7
research                       6
actor                          6
student                        6
financial                      6
Logistics                      5
accountant                     5
ceo                            5
communication                  4
architect                      3
writer                         3
photog

In [337]:
# Clear invalid values in the "job" column
df["job"] = df["job"].fillna("empty")  # Fill NaN values with an empty string
df["job"] = df["job"].astype(str)  # Convert all values to text strings

# Apply the function to categorize activities
df["job"] = df["job"].apply(lambda x: categorize_activities(x, dict_job))

In [338]:
df.head()

,name,age,location,about_me,job,education,height,intentions,exercise,educationTag,drinking,smoking,childrens,religion,politics,gender,zodiacSign,profile
0,Tarek,26,Sant Cugat del Valles,NaN,Other,NaN,175,Something casual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,woman
1,Guillem,41,Barcelona,Philosophy and film studies.\n\nFrom electroni...,film industry,UAB-UB-EDC,184,Something casual,Sometimes,Graduate degree,Socially,Socially,Not sure yet,Atheist,Left,NaN,NaN,woman
2,Bernardo,31,Barcelona,NaN,Other,NaN,180,Something casual,Sometimes,Graduate degree,Socially,Socially,Not sure yet,Atheist,Left,Man,NaN,woman
3,Leonardo,28,Barcelona,Looking for company to explore the World,Other,UFSC,174,NaN,Sometimes,Undergraduate degree,Socially,Never,Not sure yet,NaN,Apolitical,Man,Aquarius,woman
4,Ahmed,31,Barcelona,NaN,Other,NaN,186,Relationship,Sometimes,Graduate degree,Socially,Socially,Not sure yet,NaN,Apolitical,Man,Scorpio,woman
